# Is a good portion of an ML execution spent in C code?

In [1]:
import pstats
import io
import pandas as pd
import numpy as np

import os
import re
import importlib

# The module below implements a function that identifies which modules are C Extensions
import c_extens # thanks https://stackoverflow.com/questions/20339053/in-python-how-can-one-tell-if-a-module-comes-from-a-c-extension

# The function below turns the binary output of cProfile into a string
We acquire the binary output of [cProfile](https://docs.python.org/3.7/library/profile.html#module-cProfile) by running:

`python -m cProfile -o output.cprof code_i_want_to_profile.py`

In [2]:
def get_raw_output(source):
    stream = io.StringIO()
    stats = pstats.Stats(source, stream=stream)
    stats.print_stats()
    raw_output = stream.getvalue().split('\n')
    stream.close()
    return raw_output

# The function below turns the string output of cProfile into a pandas DF

In [3]:
def to_df(raw: str) -> pd.DataFrame:
    columns = ['ncalls', 'tottime', 'percall', 'cumtime', 'percall', 'filename:lineno(function)']
    
    # filter non-tabular metadata
    tabular_raw = None
    for i, line in enumerate(raw):
        if line.split() == columns:
            tabular_raw = raw[i:]
            break
    
    # we unique the names of the columns for Pandas
    columns[2] = 'percallnode'
    columns[4] = 'percallgraph'
    
    # we rename for pandas
    columns[5] = 'plf' # p: path, l: lineno, f: function
    
    import re
    tabular = [re.split('\s\s+', line) for line in tabular_raw[1:]]
    
    # preprocess
    for i in range(len(tabular)):
        # filter empty cells
        line = tabular.pop(0)
        line = [word for word in line if word]
        if line:
            # Last cell needs to be unmerged
            last = line.pop()
            split_idx = last.find(' ')
            n, s = last[0:split_idx], last[split_idx+1:]
            line.append(n)
            line.append(s)
            assert len(line) == len(columns), "i: {}, len: {} --- {}".format(i, len(line), line)
            tabular.append(line)
    
    # Ready to load into DF
    df = pd.DataFrame(tabular)
    df.columns = columns
    return df

# Load the cProfile binary into a pandas DataFrame

In [4]:
df = to_df(get_raw_output('basic.cprof'))

In [5]:
df

,ncalls,tottime,percallnode,cumtime,percallgraph,plf
0,825,0.000,0.000,0.000,0.000,{method 'find' of 'bytearray' objects}
1,58,0.000,0.000,0.000,0.000,{method 'translate' of 'bytearray' objects}
2,13,0.000,0.000,0.000,0.000,{method 'decode' of 'bytes' objects}
3,2,0.000,0.000,0.000,0.000,{built-in method maketrans}
4,1,0.000,0.000,0.000,0.000,{method 'rfind' of 'bytes' objects}
5,1,0.000,0.000,0.000,0.000,{method 'rstrip' of 'bytes' objects}
6,7,0.000,0.000,0.000,0.000,{method 'startswith' of 'bytes' objects}
7,430,0.000,0.000,0.000,0.000,{method 'get' of 'mappingproxy' objects}
8,616,0.000,0.000,0.000,0.000,{method 'values' of 'mappingproxy' objects}
9,96,0.000,0.000,0.000,0.000,{method 'items' of 'mappingproxy' objects}


# Now I need to preprocess the plf column to extract the relevant modules from the text

Do you see text like `{built-in method torch._C._get_tracing_state}`? I can't use that. I need to transform it so I extract the module name and function name from such strings. The template class below is like a cookie cutter. You press it down on a string and the remainder is the module name and function name (two-tuple).

In [14]:
class Template:
    
    def __init__(self):
        self.special = {
            "\{method '.*' of '.*' objects\}": lambda x: (x.split("'")[3], x.split("'")[1]),
            "\{built-in method .*\}": lambda x: (None, x[1:-1].split()[-1]),
            "\{function (.*)\.(.*) at 0x(\w)*\}": lambda x: (x[1:-1].split()[1].split('.')),
            "<frozen (.*)\.(.*)>:(\d+)\(\w*\)": lambda x: (x.split(':')[0][len('<frozen') + 1:-1], x.split(':')[1].split('(')[1][:-1]),
            "<frozen (.*)\.(.*)>:(\d+)\(<\w*>\)": lambda x: (x.split(':')[0][len('<frozen') + 1:-1], None),
            "\w+\.py:\d+\(\w+\)": lambda x: (None, None),
            "\w+\.py:\d+\(<\w+>\)": lambda x: (None, None),
            "<string>:\d+\(<\w+>\)": lambda x: (None, None),
            "<string>:\d+\(\w+\)": lambda x: (None, None),
            "\{instance\}": lambda x: (None, None)
        }
        
    def match(self, t):
        for k in self.special:
            pattern = re.compile(k)
            if pattern.match(t):
                return k
            
    def is_c_extension(self, t):
        k = self.match(t)
        if k is not None:
            m, f = self.special[k](t) # m: module, f: function
            if m is None:
                return False
            ms = m.split('.')
            for i in range(len(ms)):
                candidate_mod_name = '.'.join(ms[:len(ms)-i])
                try:
                    candidate_mod = importlib.import_module(candidate_mod_name)
                except:
                    continue
                try:
                    ans = c_extens.is_c_extension(candidate_mod) # I am trusting a stackoverflow solution
                    if ans is None:
                        ans = False
                    return ans
                except:
                    return False
        return False
        

toby = Template()

# We see calls to C Extensions below

In [19]:
c_df = df[df['plf'].map(toby.is_c_extension)]
c_df

,ncalls,tottime,percallnode,cumtime,percallgraph,plf
349,1,0.000,0.000,0.000,0.000,{method 'disable' of '_lsprof.Profiler' objects}
1048,16,0.000,0.000,0.000,0.000,{method 'getrandbits' of '_random.Random' obje...
1309,60,0.005,0.000,0.005,0.000,{method 'run_backward' of 'torch._C._EngineBas...
3559,62,0.000,0.000,0.000,0.000,{method 'dim' of 'torch._C._TensorBase' objects}
3560,12,0.000,0.000,0.000,0.000,{method 'item' of 'torch._C._TensorBase' objects}
3561,124,0.000,0.000,0.000,0.000,{method 'size' of 'torch._C._TensorBase' objects}
3562,120,0.001,0.000,0.001,0.000,{method 'add_' of 'torch._C._TensorBase' objects}
3563,118,0.000,0.000,0.000,0.000,{method 'detach_' of 'torch._C._TensorBase' ob...
3564,60,0.000,0.000,0.000,0.000,{method 'numel' of 'torch._C._TensorBase' obje...
3565,60,0.000,0.000,0.000,0.000,{method 't' of 'torch._C._TensorBase' objects}


# Approximately what fraction of time was spent in C code?

In [25]:
"{0:.3%}".format(c_df['cumtime'].map(float).sum() / df['cumtime'].map(float).max())

'0.653%'